In [28]:
import torch
from pandas_datareader import data
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd
import joblib
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from models.LSTM import LSTM
#from utils.split_data import split_data
from sklearn.ensemble import RandomForestRegressor
from torch.nn import GRU
import matplotlib.pyplot as plt
import io
from itertools import product

def split_data(stock, lookback):
    data_raw = stock
    data = []
    for index in range(len(data_raw) - lookback):
        data.append(data_raw[index: index + lookback])
        
    data = np.array(data)
    test_set_size = int(np.round(0.2*stock.shape[0]))
    train_set_size = stock.shape[0] - test_set_size

    x_train = data[:train_set_size, :-1,:]
    y_train = data[:train_set_size, -1, :]

    x_test = data[train_set_size:, :-1, :]
    y_test = data[train_set_size:, -1, :]
    
    return (x_train, y_train, x_test, y_test)

datapath = 'stock_data/0066.HK.csv'
startdate = "2022-01-01"
enddate = "2024-3-30"

 #####读取数据
data_to_be_predict = pd.read_csv(datapath)
data_to_be_predict['Date'] = pd.to_datetime(data_to_be_predict['Date'])
data_to_be_predict = data_to_be_predict.set_index('Date')
data_to_be_predict = data_to_be_predict.loc[startdate:enddate]

yf.pdr_override()
scaler = MinMaxScaler(feature_range=(-1, 1))  

print(data_to_be_predict)

                 Open       High        Low      Close  Adj Close   Volume
Date                                                                      
2022-01-03  41.950001  42.349998  41.900002  42.150002  39.278328  1761872
2022-01-04  42.099998  42.349998  41.500000  42.299999  39.418106  2910078
2022-01-05  42.349998  42.799999  42.250000  42.700001  39.790859  4416855
2022-01-06  42.750000  42.799999  42.000000  42.349998  39.464703  1457827
2022-01-07  42.250000  42.500000  41.900002  42.500000  39.604481  2066317
...               ...        ...        ...        ...        ...      ...
2024-03-22  25.900000  25.950001  25.299999  25.450001  25.450001  3367528
2024-03-25  25.500000  25.750000  25.250000  25.600000  25.600000  2579997
2024-03-26  26.000000  26.299999  25.600000  26.150000  26.150000  5201511
2024-03-27  26.100000  26.350000  25.850000  26.150000  26.150000  4277108
2024-03-28  26.200001  26.299999  25.700001  25.799999  25.799999  3276381

[550 rows x 6 columns]


In [19]:

    which_model = 'lstm'

    lookback = 50
    # Open = data_to_be_predict['Open'].values.reshape(-1,1)
    data_close = scaler.fit_transform(data_to_be_predict['Close'].values.reshape(-1,1))
                
    x_train, y_train, x_test, y_test = split_data(data_close, lookback)

    x_test = torch.from_numpy(x_test).type(torch.Tensor)
    y_test_lstm = torch.from_numpy(y_test).type(torch.Tensor)
    x_train = torch.from_numpy(x_train).type(torch.Tensor)
    y_train_lstm = torch.from_numpy(y_train).type(torch.Tensor)

    model = torch.load('pthfile/lstm_model_complete.pth')

    data_close = scaler.inverse_transform(data_close)

    test_pred = model(x_test)
    test_pred = pd.DataFrame((test_pred.detach().numpy()))
    train_pred = model(x_train)
    train_pred = pd.DataFrame((train_pred.detach().numpy()))

    trainPredictPlot=np.empty_like(data_close)
    trainPredictPlot[:,:]=np.nan
    trainPredictPlot[lookback:len(scaler.inverse_transform(train_pred))+lookback] = scaler.inverse_transform(train_pred)

    testPredictPlot=np.empty_like(data_close)
    testPredictPlot[:,:] = np.nan
    testPredictPlot[len(train_pred)+(lookback):len(data_close)] = scaler.inverse_transform(test_pred)

    ####整体predict序列
    whole_predict = np.empty_like(data_close)
    whole_predict[:,:] = np.nan
    whole_predict[lookback:len(scaler.inverse_transform(train_pred))+lookback] = scaler.inverse_transform(train_pred)
    whole_predict[len(train_pred)+(lookback):len(data_close)] = scaler.inverse_transform(test_pred)
    # actual = data_close[lookback:]

    whole_predict = pd.DataFrame({'Date':data_to_be_predict.index,'pred_close':whole_predict.flatten(),'act_close':data_to_be_predict['Close'],'open':data_to_be_predict['Open']})

    trainPredictPlot = pd.DataFrame({'Date':data_to_be_predict.index,'Close':trainPredictPlot.flatten()})
    testPredictPlot = pd.DataFrame({'Date':data_to_be_predict.index,'Close':testPredictPlot.flatten()})
    data_close = pd.DataFrame({'Date':data_to_be_predict.index,'Close':data_close.flatten()})

    mse_test = mean_squared_error(y_test_lstm.detach().numpy(),test_pred)
    mse_train = mean_squared_error(y_train_lstm.detach().numpy(),train_pred)

    #return mse_train,mse_test,testPredictPlot,trainPredictPlot,data_close,whole_predict


In [20]:

    which_model = 'linearregression'

    # stock_data = data.get_data_yahoo('0066.HK', start="2010-01-01", end="2020-06-30").reset_index()
    data_to_be_predict_shift = data_to_be_predict.shift(1)
    data_to_be_predict_shift = data_to_be_predict_shift.iloc[1:]
    data_to_be_predict = data_to_be_predict.iloc[1:]
    predict_x = data_to_be_predict_shift[['Open', 'High', 'Low', 'Volume','Close']]
    predict_y = data_to_be_predict['Close']

    # print(predict_x)
    predict_y = predict_y.sort_index()
    predict_x = predict_x.sort_index()
    model = joblib.load('pthfile/linear_regression_model.joblib')
    predict_result = model.predict(predict_x)
    mse = mean_squared_error(predict_y, predict_result)

    pred = pd.DataFrame({'Date':pd.to_datetime(predict_x.index),'Close':predict_result})
    actual = pd.DataFrame({'Date':pd.to_datetime(predict_x.index),'Close':predict_y.values})

    whole_predict = pd.DataFrame({'Date':data_to_be_predict.index,'pred_close':predict_result.flatten(),'open':data_to_be_predict['Open'],'act_close':data_to_be_predict['Close']})
    ###trading 在开盘时，如果预测今天的close比开盘价高则买入

    #return mse,actual,pred,whole_predict

In [21]:

    which_model = 'randomforest'
   
    # stock_data = data.get_data_yahoo('0066.HK', start="2010-01-01", end="2020-06-30").reset_index()
    lookback = 600               
        
    mtr = scaler.fit_transform(data_to_be_predict.values)
    # 构建训练集和测试集
    x_train, y_train, x_test, y_test = [], [], [], []
    for i in range(len(mtr) - lookback):
        x_train.append(mtr[i:i+lookback])
        y_train.append(mtr[i+lookback])

    for i in range(len(mtr) - lookback, len(mtr)):
        x_test.append(mtr[i-lookback:i])
        y_test.append(mtr[i])

    # 转换为 numpy 数组并重塑为二维数组
    x_train, y_train, x_test, y_test = np.array(x_train), np.array(y_train), np.array(x_test), np.array(y_test)
    x_train = x_train.reshape(x_train.shape[0], -1)
    x_test = x_test.reshape(x_test.shape[0], -1)
    # 构建训练集和测试集
   
    # Load pre-trained Random Forest model
    model = torch.load('pthfile/randomforest_model.pth')
        
    #model = RandomForestRegressor(n_estimators=100, random_state=42)
    #model.fit(x_train, y_train.ravel())

    train_pred = model.predict(x_train)
    test_pred = model.predict(x_test)
        
    mse_train = mean_squared_error(y_train, train_pred)
    mse_test = mean_squared_error(y_test, test_pred)
        
    whole_predict = pd.DataFrame({'Date':data_to_be_predict.index, 'pred_close': np.concatenate([train_pred, test_pred]), 'open':data_to_be_predict['Open'],'act_close':data_to_be_predict['Close']})
    
    #return mse_train, mse_test, y_train, train_pred, y_test, test_pred, mtr, whole_predict
    #return mse_train, mse_test, y_train, train_pred, y_test, test_pred, mtr


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (600,) + inhomogeneous part.

In [31]:
lookback = 600

mtr = scaler.fit_transform(data_to_be_predict.values)

# 构建训练集和测试集
x_train, y_train, x_test, y_test = [], [], [], []
for i in range(len(mtr) - lookback):
    x_train.append(mtr[i:i+lookback])
    y_train.append(mtr[i+lookback])

for i in range(len(mtr) - lookback, len(mtr)):
    x_test.append(mtr[i-lookback:i])
    y_test.append(mtr[i])

x_train = np.vstack(x_train)
y_train = np.vstack(y_train)
x_test = np.vstack(x_test)
y_test = np.vstack(y_test)

# 转换为 numpy 数组并重塑为三维数组
x_train = np.array(x_train).reshape(-1, lookback, 1)
y_train = np.array(y_train)
x_test = np.array(x_test).reshape(-1, lookback, 1)
y_test = np.array(y_test)

# Load pre-trained Random Forest model
model = torch.load('pthfile/randomforest_model.pth')

# 进行预测
train_pred = model.predict(x_train)
test_pred = model.predict(x_test)

mse_train = mean_squared_error(y_train, train_pred)
mse_test = mean_squared_error(y_test, test_pred)

# 构建整体预测的 DataFrame
whole_predict = pd.DataFrame({'Date': data_to_be_predict.index,
                              'pred_close': np.concatenate([train_pred, test_pred]),
                              'open': data_to_be_predict['Open'],
                              'act_close': data_to_be_predict['Close']})

mse_train, mse_test, y_train, train_pred, y_test, test_pred, mtr, whole_predict

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (600,) + inhomogeneous part.

In [23]:

    which_model = 'svm'
    
    lookback = 100   
        
    mtr = scaler.fit_transform(data_to_be_predict.values)
    # 构建训练集和测试集
    x_train, y_train, x_test, y_test = [], [], [], []
    for i in range(len(mtr) - lookback):
        x_train.append(mtr[i:i+lookback])
        y_train.append(mtr[i+lookback])
    for j in range(len(mtr) - lookback, len(mtr)):
        x_test.append(mtr[j-lookback:j])
        y_test.append(mtr[j])

    # 转换为 numpy 数组并重塑为二维数组
    x_train, y_train, x_test, y_test =  np.array(x_train), np.array(y_train), np.array(x_test), np.array(y_test)
    x_train = x_train.reshape(x_train.shape[0], -1)
    x_test = x_test.reshape(x_test.shape[0], -1)

    # Load pre-trained SVM model
    model = torch.load('pthfile/svm_model.pth')

    # 进行预测
    train_pred = model.predict(x_train)
    test_pred = model.predict(x_test)
        
    # 计算均方误差
    mse_train = mean_squared_error(y_train, train_pred)
    mse_test = mean_squared_error(y_test, test_pred)

    # 绘制预测结果
    plt.plot(y_train, label='Actual Train')
    plt.plot(train_pred, label='Predicted Train')
    plt.legend()
    plt.title('SVM Predictions on Train Data')
    plt.show()
        
    plt.plot(y_test, label='Actual Test')
    plt.plot(test_pred, label='Predicted Test')
    plt.legend()
    plt.title('SVM Predictions on Test Data')
    plt.show()
        
    whole_predict = pd.DataFrame({'Date':data_to_be_predict.index, 'pred_close': np.concatenate([train_pred, test_pred]), 'open':data_to_be_predict['Open'],'act_close':data_to_be_predict['Close']})

    #return mse_train, mse_test, y_train, train_pred, y_test, test_pred, mtr, whole_predict


C:\Users\patty.hao\AppData\Local\anaconda3\envs\nlp_env\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator SVR from version 1.0.2 when using version 1.5.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


ValueError: X has 600 features, but SVR is expecting 100 features as input.

In [8]:
def predict_gru(datapath = 'stock_data/0066.HK.csv',startdate = "2022-01-01",enddate = "2024-3-30"):
    which_model == 'gru'
    #####读取数据
    data_to_be_predict = pd.read_csv(datapath)
    data_to_be_predict['Date'] = pd.to_datetime(data_to_be_predict['Date'])
    data_to_be_predict = data_to_be_predict.set_index('Date')
    data_to_be_predict = data_to_be_predict.loc[startdate:enddate]

    yf.pdr_override()
    scaler = MinMaxScaler(feature_range=(-1, 1))  
    # stock_data = data.get_data_yahoo('0066.HK', start="2010-01-01", end="2020-06-30").reset_index()
    # 数据准备
    def prepare_data(data, lookback):
        x, y = [], []
        for i in range(len(data) - lookback):
            x.append(data[i:(i + lookback)])
            y.append(data[i + lookback])
        return np.array(x), np.array(y)

    # 加载数据
    csv_file_path = 'SSE_Index.csv'
    df = pd.read_csv(csv_file_path)
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date', inplace=True)

    # 数据预处理
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaled_data = scaler.fit_transform(df.values)

    # 定义超参数
    lookback = 100

    # 加载预训练的GRU模型
    loaded_model = torch.jit.load('pthfile/gru_model_script.pth')

    # 准备测试集
    train_data_scaled = scaler.transform(df.values)
    x_train, y_train = prepare_data(train_data_scaled, lookback)
    x_train = torch.from_numpy(x_train).type(torch.Tensor)  # 将 ndarray 转换为 Tensor
        
    test_data_scaled = scaler.transform(df.values)
    x_test, y_test = prepare_data(test_data_scaled, lookback)
    x_test = torch.from_numpy(x_test).type(torch.Tensor)  # 将 ndarray 转换为 Tensor

    # 使用加载的模型进行预测
    loaded_model.eval()
    train_pred = loaded_model(x_train)
    train_pred = scaler.inverse_transform(train_pred.detach().numpy())
    test_pred = loaded_model(x_test)
    test_pred = scaler.inverse_transform(test_pred.detach().numpy())

    # 可视化预测结果
    # 绘制训练集预测结果
    plt.figure(figsize=(12, 6))
    plt.plot(df.index[lookback:len(train_pred) + lookback], df['Close'][lookback:len(train_pred) + lookback], label='Actual Train')
    plt.plot(df.index[lookback:len(train_pred) + lookback], train_pred, label='Predicted Train')
    plt.legend()
    plt.title('GRU Predictions on Train Data')
    plt.show()

        
    plt.figure(figsize=(12, 6))
    plt.plot(df.index[lookback:], df['Close'][lookback:], label='Actual Test')
    plt.plot(df.index[lookback:], test_pred, label='Predicted Test (GRU Model)')
    plt.legend()
    plt.title('GRU Predictions on Test Data')
    plt.show()

    # 计算均方误差
    mse_train = mean_squared_error(y_train, train_pred)
    mse_test = mean_squared_error(y_test, test_pred)#df['Close'][lookback:]
    # print('MSE ON TRAIN:', mse_train)
    # print('MSE ON TEST:', mse_test) 
        
    whole_predict = pd.DataFrame({'Date':data_to_be_predict.index, 'pred_close': np.concatenate([train_pred, test_pred]), 'open':data_to_be_predict['Open'],'act_close':data_to_be_predict['Close']})

    return mse_train, mse_test, y_train, train_pred, y_test, test_pred, scaled_data, whole_predict